In [1]:
import tensorflow as tf 
from basemodel import LeNet, LeNetEncoder, Discriminator
from sklearn.metrics import accuracy_score
from dataloader import DataLoader

## Lenet Model For Source 

In [2]:
model = LeNet()

In [3]:
sample_input = tf.random.uniform(shape=(10, 28, 28, 1))
y = model(sample_input)

In [4]:
model.summary()

Model: "LeNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 LeNetEncoder (LeNetEncoder)  multiple                 926070    
                                                                 
 LeNetClassifier (LeNetClass  multiple                 5010      
 ifier)                                                          
                                                                 
Total params: 931,080
Trainable params: 931,080
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.encoder.summary()

Model: "LeNetEncoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  520       
                                                                 
 conv2d_1 (Conv2D)           multiple                  25050     
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 dense (Dense)               multiple                  900500    
                                                                 
 dropout (Dropout)           multiple                 

In [6]:
model.classifier.summary()

Model: "LeNetClassifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  5010      
                                                                 
Total params: 5,010
Trainable params: 5,010
Non-trainable params: 0
_________________________________________________________________


## Prepare Datasets (source)

DataLoder module loads the MNIST and USPS datasets and splits them into train test

In [7]:
loader = DataLoader("mnist")

Loading dataset mnist


In [8]:
(x_train, y_train) = loader.getData('train')
(x_test, y_test) = loader.getData('test')

Creating Data Iterator for mini-batch training loop

In [9]:
BATCH_SIZE = 64

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [11]:
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

## Train on source with cross-entopy loss

In [12]:
loss_metric = tf.keras.metrics.Mean()
m = tf.keras.metrics.Accuracy()

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
ce_loss_fn = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

In [14]:
epochs = 10    

In [16]:
# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))
    
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch) in enumerate(train_dataset):
        #x_batch_train = tf.expand_dims(x_batch_train, axis=3)
        with tf.GradientTape() as tape:
            pred = model(x_batch_train)
                
            loss = tf.reduce_mean(ce_loss_fn(y_batch, pred))
            m.update_state(tf.argmax(y_batch, 1),tf.argmax(pred, 1))
    
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
        loss_metric(loss)
    
        if step % 100 == 0:
            print("step %d: mean loss = %.4f acc=%.4f" % (step, loss_metric.result(),  m.result()))
        
    val_loss_metric = tf.keras.metrics.Mean()
    val_acc = tf.keras.metrics.Accuracy()
        
    for i, (x_batch_test, y_batch_test) in enumerate(test_dataset):
        #x_batch_test = tf.expand_dims(x_batch_test, axis=3)
        pred_val = model(x_batch_test)                
        val_loss = tf.reduce_mean(ce_loss_fn(y_batch_test, pred_val))
        val_loss_metric(val_loss)
        val_acc.update_state(tf.argmax(y_batch_test, 1),tf.argmax(pred_val, 1))
    print("Validation: Mean loss = %.4f acc=%.4f" % (val_loss_metric.result(),  val_acc.result()))                    
    

Start of epoch 0
step 0: mean loss = 0.5677 acc=0.8284
step 100: mean loss = 0.2717 acc=0.9188


KeyboardInterrupt: 

In [17]:
#Save the model
#model.save_weights('./models/mnist_lenet')

## Training on Target Domain (assuming NO labels available)

Here labels will be used only for evaluation of results NOT for training

In [18]:
usps_loader = DataLoader("usps")
X_tr, y_tr = usps_loader.getData('train')

usps_loader = DataLoader("usps")
X_tr_test, y_tr_test = usps_loader.getData('test')

mnist_loader = DataLoader("mnist")
X_sr, y_sr = mnist_loader.getData("train")

Loading dataset usps
Loading dataset usps
Loading dataset mnist


Creating Dataset Iterators for mini-batch training loop on target 

In [19]:
BATCH_SIZE_TR = 50

In [20]:
tgt_dataset = tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
tgt_dataset = tgt_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE_TR)

src_dataset = tf.data.Dataset.from_tensor_slices((X_sr, y_sr))
src_dataset = src_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE_TR)

tgt_dataset_test = tf.data.Dataset.from_tensor_slices((X_tr_test, y_tr_test))
tgt_dataset_test = tgt_dataset_test.shuffle(buffer_size=1024).batch(BATCH_SIZE_TR)

## Load model 

In [21]:
targetModel = LeNet(training = False)

In [22]:
sample_input = tf.random.uniform(shape=(10, 28, 28, 1))
y = targetModel(sample_input)

In [23]:
print(targetModel.encoder.layers[0].weights[0][0])

tf.Tensor(
[[[-0.2952079   0.37245205 -0.03149784  0.37271476 -0.25152418
   -0.3970836   0.02068714 -0.19946045 -0.38615113  0.2085048
    0.15617841 -0.26530036 -0.37290442  0.08710288  0.46271327
    0.19992392  0.48179376  0.04985712  0.22105566 -0.4961837 ]]

 [[ 0.3154899  -0.02073991  0.35343343 -0.08006986  0.16421616
    0.53407896 -0.10781709  0.20271663  0.21474242 -0.3815879
   -0.11711562  0.11146358  0.23061554 -0.4468818  -0.06483935
    0.08043522 -0.49446213 -0.01319749 -0.18818846  0.15906008]]

 [[-0.0286163  -0.02196479 -0.4462118   0.01093923  0.07458931
   -0.22513442  0.00879162  0.18701689  0.33425054 -0.15878253
   -0.08912595 -0.20571202  0.15419707 -0.2604865  -0.02617104
    0.00968052 -0.44290906 -0.06332861 -0.32153285 -0.00880899]]

 [[-0.01120962  0.19201182  0.20417696  0.18580563 -0.11774364
   -0.2500204   0.4080259   0.25073114 -0.3796424   0.5824057
    0.349913    0.5214375   0.5937191  -0.3679185   0.16061176
    0.04294679  0.05984076 -0.4945863 

In [24]:
checkpoint_path = ".\\models\\mnist_lenet"
# init weights of target encoder with those of source encoder
targetModel.load_weights(checkpoint_path)  

Check if new weights loaded properly

In [25]:
print(targetModel.encoder.layers[0].weights[0][0])

tf.Tensor(
[[[ 0.07447814 -0.00880726 -0.2455518  -0.14313318 -0.206044
    0.01054084 -0.45289755  0.39708784 -0.06829995  0.41723385
    0.07748751 -0.42984262 -0.03139526 -0.11600942 -0.0976113
    0.10573574 -0.37824258  0.02161951  0.5805371  -0.07381678]]

 [[-0.03983057  0.1241329   0.2847217  -0.24746835 -0.27586755
   -0.05461131 -0.09643356 -0.09678437  0.19254015 -0.3950874
   -0.20059995 -0.64347035 -0.6227867  -0.05109233  0.00334542
    0.02967518 -0.3179615   0.0135891   0.24304534 -0.19977543]]

 [[-0.0078422   0.06580239 -0.3811191  -0.18182144  0.36455932
   -0.0654147   0.314523    0.3750171   0.10549959 -0.09135032
   -0.35417858 -0.27474433  0.2563004  -0.37529317 -0.10289144
    0.42643112 -0.34611034  0.28337616 -0.27924997 -0.3395373 ]]

 [[ 0.2836141  -0.49414167 -0.13555625  0.09657162  0.15958536
   -0.12617801  0.3260055  -0.02024609  0.6486877   0.13205835
   -0.15898919 -0.12551579 -0.1586223   0.09797388 -0.5715533
   -0.39811888  0.03213071 -0.00512786 -

### Check Accuracy of Target Domain without any further training

In [26]:
ypred = targetModel(X_tr)        
predClass = tf.argmax(ypred, axis=1)
print(accuracy_score(y_tr, predClass))

0.8302016184336853


In [27]:
ypred = targetModel(X_tr_test)        
predClass = tf.argmax(ypred, axis=1)
print(accuracy_score(y_tr_test, predClass))

0.8086696562032885


# Domain Adaptation for Target Domain

Load source model as is

In [28]:
model = LeNet(training=False)
checkpoint_path = ".\\models\\mnist_lenet"
model.load_weights(checkpoint_path)         
         
ypred = model(X_tr)        
predClass = tf.argmax(ypred, axis=1)
print(accuracy_score(y_tr, predClass))

0.8302016184336853


In [30]:
discriminator = Discriminator(500)
y_disc = discriminator(targetModel.encoder(sample_input))
discriminator.summary()

Model: "Discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             multiple                  250500    
                                                                 
 dense_10 (Dense)            multiple                  250500    
                                                                 
 dense_11 (Dense)            multiple                  501       
                                                                 
Total params: 501,501
Trainable params: 501,501
Non-trainable params: 0
_________________________________________________________________


## Adverserial Losses

In [31]:
def discriminator_loss(source_output, target_output):
    
    pred = tf.concat([source_output, target_output], axis =0)
    label = tf.concat([tf.ones_like(source_output), tf.zeros_like(target_output)], axis=0)
    loss = cross_entropy(label, pred)
    return tf.reduce_mean(loss)

def generator_loss(target_output):
    return tf.reduce_mean(cross_entropy(tf.ones_like(target_output), target_output))


In [32]:
discrinator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
targetEncoder_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
cross_entropy = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)


In [35]:
# init weights of target encoder with those of source encoder
targetModel = LeNet()
targetModel.load_weights(checkpoint_path)         
targetEncoder = targetModel.encoder

## Adverserial Training Loop

In [36]:
num_epochs = 100
gen_loss_metric = tf.keras.metrics.Mean()
dis_loss_metric = tf.keras.metrics.Mean()
target_acc = tf.keras.metrics.Accuracy()

for epoch in range(num_epochs):
    gen_loss_metric.reset_states()
    dis_loss_metric.reset_states()
    target_acc.reset_states()
    
    print("Epoch {}".format(epoch))
    tgt_dataset = tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
    tgt_dataset = tgt_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE_TR)

    src_dataset = tf.data.Dataset.from_tensor_slices((X_sr, y_sr))
    src_dataset = src_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE_TR)

    tgt_dataset_test = tf.data.Dataset.from_tensor_slices((X_tr_test, y_tr_test))
    tgt_dataset_test = tgt_dataset_test.shuffle(buffer_size=1024).batch(BATCH_SIZE_TR)
    data_zip = enumerate(zip(src_dataset, tgt_dataset))
    
    for step, ((images_src, _), (images_tgt, y_tgt)) in data_zip:
        #print(step, images_src.shape, images_tgt.shape)
        if images_tgt.shape[0]<50:
            break
        with tf.GradientTape() as disc_tape:
            #train discriminator
            target_encoded = targetEncoder(images_tgt)
            source_encoded = model.encoder(images_src)
            
            target_disc_pred = discriminator(target_encoded)
            source_disc_pred = discriminator(source_encoded)
            
            disc_loss = discriminator_loss(source_disc_pred, target_disc_pred)
        gradients_of_D = disc_tape.gradient(disc_loss, discriminator.trainable_weights)
        discrinator_optimizer.apply_gradients(zip(gradients_of_D, discriminator.trainable_weights))
        dis_loss_metric(disc_loss)
        
        with tf.GradientTape() as gen_tape:     
            #train generator            
            target_encoded = targetEncoder(images_tgt)
            target_disc_pred = discriminator(target_encoded)
            gen_loss = generator_loss(target_disc_pred)
        
        gradients_of_G = gen_tape.gradient(gen_loss, targetEncoder.trainable_weights)
        targetEncoder_optimizer.apply_gradients(zip(gradients_of_G, targetEncoder.trainable_weights))            
        gen_loss_metric(gen_loss)
        
            
        if step % 20 == 0:
            print("epoch %d step %d :Mean gen loss = %.4f Disc Loss=%.4f" % (epoch, 
                                                                             step,
                                                              gen_loss_metric.result(),
                                                              dis_loss_metric.result()))
                
            for i, (x_batch_test, y_batch_test) in enumerate(tgt_dataset_test):
                #test_predictions
                pred = targetModel(x_batch_test)
                predClass = tf.argmax(pred, axis=1)                    
                target_acc.update_state(y_batch_test,predClass)
            print("Target Acc:%.4f" %(target_acc.result()))
            
    print("Epoch {} Acc{}".format(epoch, target_acc.result()))



Epoch 0
epoch 0 step 0 :Mean gen loss = 0.7130 Disc Loss=0.7669
Target Acc:0.8087
epoch 0 step 20 :Mean gen loss = 0.7219 Disc Loss=0.5748
Target Acc:0.8084
epoch 0 step 40 :Mean gen loss = 0.8231 Disc Loss=0.5093
Target Acc:0.8102
epoch 0 step 60 :Mean gen loss = 0.8723 Disc Loss=0.4818
Target Acc:0.8130
epoch 0 step 80 :Mean gen loss = 0.8882 Disc Loss=0.4713
Target Acc:0.8151


KeyboardInterrupt: 

In [ ]:
#targetEncoder.save_weights('./models/target_encoder/adda_usps_lenet')                

# Results of ADDA

In [43]:
# init weights of target encoder with those of source encoder
targetModel = LeNet()
targetModel.load_weights(checkpoint_path)         
targetEncoder = targetModel.encoder

In [44]:
targetEncoder.load_weights('./models/target_encoder/adda_usps_lenet')

Evaluate Train Accuracy

In [46]:
target_acc = tf.keras.metrics.Accuracy()
for i, (x_batch_test, y_batch_test) in enumerate(tgt_dataset):
    #train_predictions
    pred = targetModel(x_batch_test)
    predClass = tf.argmax(pred, axis=1)                    
    target_acc.update_state(y_batch_test,predClass)
print("Target Test Acc:%.4f" %(target_acc.result()))

Target Test Acc:0.9671


Evaluate Test Accuracy

In [47]:
target_acc = tf.keras.metrics.Accuracy()
for i, (x_batch_test, y_batch_test) in enumerate(tgt_dataset_test):
    #test_predictions
    pred = targetModel(x_batch_test)
    predClass = tf.argmax(pred, axis=1)                    
    target_acc.update_state(y_batch_test,predClass)
print("Target Test Acc:%.4f" %(target_acc.result()))

Target Test Acc:0.9542
